In [ ]:
from data import store
import pandas as pd
import numpy as np

In [ ]:
train = store['train']
train.head()

In [ ]:
dates = train['clickTime']
dates = pd.DataFrame(dates//10000)
dategrp = dates.groupby(by='clickTime').groups

In [ ]:
# some statistics
clickCount={}
convertCount={}
convertRate={}
usersCount={}

labels = train['label']
users = train['userID']
for d in dategrp:
    lbl = labels.iloc[dategrp[d]]
    clickCount[d] = len(lbl)
    convertCount[d] = np.sum(lbl)
    convertRate[d] = convertCount[d]*1.0/clickCount[d]
    
    usrs = users.iloc[dategrp[d]]
    usersCount[d] = len(users.value_counts())

In [ ]:
days = dategrp.keys()
days

In [ ]:
def plotTable(tbl,mark='o'):
    x = days
    y = [tbl[d] for d in x]
    plot(x,y,mark)

In [ ]:
%pylab inline
plotTable(clickCount,'ro')
clickCount

In [ ]:
plotTable(convertCount,'ro')
convertCount

In [ ]:
plotTable(convertRate,'ro')
convertRate

In [ ]:
plotTable(usersCount,'ro')
usersCount

In [ ]:
plot(days,np.array(days)%7,'o')

# 结论
- 18，19，20号，点击量非常低。
- 19号转化率非常高
- 30号转化率非常低 (而一个星期前的23号，一切都很正常。原因？未知)


因此，划分CV的时候，测试集要尽量避开18，19，20以及30号

In [ ]:
# 特征提取的时候使用的简化数据集
split_grps = [
    ([17,18],[19]),
    ([21,22],[23]),
#    ([25,26,27,28],[29,30]),
]
cv = []
cvgrp = []
for tr_grp, te_grp in split_grps:
    train_idx = []
    for t in tr_grp:
        train_idx.extend(dategrp[t])
    test_idx = []
    for t in te_grp:
        test_idx.extend(dategrp[t])
    cv.append((train_idx,test_idx))
    cvgrp.append((tr_grp,te_grp))

dfcv = pd.DataFrame(cv)
store['cv_simple'] = dfcv
dfcv

In [ ]:
store.flush(fsync=True)

In [ ]:
# 特征提取的时候使用的简化数据集
split_grps = [
    ([17,18,19,20,21,22,23],[24]),
    ([20,21,22,23,24,25,26],[27]),
    ([17,18,19,20,21,22,23,24,25,26,27],[28,29]),
    ([18,19,20,21,22,23,24,25,26,27,28],[29,30]),
]
cv = []
cvgrp = []
for tr_grp, te_grp in split_grps:
    train_idx = []
    for t in tr_grp:
        train_idx.extend(dategrp[t])
    test_idx = []
    for t in te_grp:
        test_idx.extend(dategrp[t])
    cv.append((train_idx,test_idx))
    cvgrp.append((tr_grp,te_grp))

dfcv = pd.DataFrame(cv)
store['cv_full'] = dfcv
dfcv

In [ ]:
store.close()